##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow Addons Networks : Sequence-to-Sequence NMT with Attention Mechanism

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/addons/tutorials/networks_seq2seq_nmt"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/addons/blob/master/docs/tutorials/networks_seq2seq_nmt.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/addons/blob/master/docs/tutorials/networks_seq2seq_nmt.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
      <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/addons/docs/tutorials/networks_seq2seq_nmt.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

## Overview
This notebook gives a brief introduction into the ***Sequence to Sequence Model Architecture***
In this noteboook you broadly cover four essential topics necessary for Neural Machine Translation:


* **Data cleaning**
* **Data preparation**
* **Neural Translation Model with Attention**
* **Final Translation with ```tf.addons.seq2seq.BasicDecoder``` and ```tf.addons.seq2seq.BeamSearchDecoder```** 

The basic idea behind such a model though, is only the encoder-decoder architecture. These networks are usually used for a variety of tasks like text-summerization, Machine translation, Image Captioning, etc. This tutorial provideas a hands-on understanding of the concept, explaining the technical jargons wherever necessary. You focus on the task of Neural Machine Translation (NMT) which was the very first testbed for seq2seq models.


## Setup

In [ ]:
!pip install tensorflow-addons==0.11.2
!pip install jiwer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from jiwer import cer

import unicodedata
import re
import numpy as np
import os
import io
import time
import pickle


## Data Cleaning and Data Preparation 

You'll use a language dataset provided by http://www.manythings.org/anki/. This dataset contains language translation pairs in the format:

---
      May I borrow this book?    ¿Puedo tomar prestado este libro?
---


There are a variety of languages available, but you'll use the English-Spanish dataset. After downloading the dataset, here are the steps you'll take to prepare the data:


1. Add a start and end token to each sentence.
2. Clean the sentences by removing special characters.
3. Create a Vocabulary with word index (mapping from word → id) and reverse word index (mapping from id → word).
5. Pad each sentence to a maximum length. (Why? you need to fix the maximum length for the inputs to recurrent encoders)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# change the input directory to your own preferences
seq_lst = pickle.load(open('drive/MyDrive/AIBuilders/seq_lst.pkl', 'rb'))

In [ ]:
seq_lst

['happy birthday พี่ เมี๊ยว ผู้ อํา นวย การสมาคม ฟ้า สี รุ้ง แห่ง ประเทศ ไทย .. ตั้งแต่ เรา รู้จัก กัน ใน วงการ hiv / aids นั้น พีท ได้รับ ความรัก ความเมตตา จาก พี่ เมี๊ยว และ พี่ ๆ ใน สมาคม ฟ้า สี รุ้ง แห่ง ประเทศ ไทย เสมอมา ... ขอบคุณ สํา หรับ การผลักดัน ให้ มีระบบ บริการ สุขภาพ ดี ดี ให้ เกิดขึ้น ในประเทศ ไทย at สมาคม ฟ้า สี รุ้ง แห่ง ประเทศ ไทย สํา นัก งานใหญ่ ราม คํา แหง rainbow sky association of thailand head office\thappy birthday พี่ เมี๊ยว ผู้ อํา นวย การสมาคม ฟ้า สี รุ้ง แห่ง ประเทศ ไทย .. ตั้งแต่ เรา รู้จัก กัน ใน วงการ hiv / aids นั้น พีท ได้รับ ความรัก ความเมตตา จาก พี่ เมี๊ยว และ พี่ ๆ ใน สมาคม ฟ้า สี รุ้ง แห่ง ประเทศ ไทย เสมอมา ... ขอบคุณ สํา หรับ การผลักดัน ให้ มีระบบ บริการ สุขภาพ ดี ๆ ให้ เกิดขึ้น ในประเทศ ไทย at สมาคม ฟ้า สี รุ้ง แห่ง ประเทศ ไทย สํา นัก งานใหญ่ ราม คํา แหง rainbow sky association of thailand head office',
 'รีวิว # thequake วินาศภัย แห่ง แผ่นดินไหว ใน นอร์เวย์ หนัง สํา หรับ คน ชอบ สาย ภัยพิบัติ ให้ 6 / 10 ที่มา ของ พล็อต เรื่อง เยี่ยม ต้นเรื่อง แอบ 

In [ ]:
len(seq_lst)

42890

In [ ]:
def download_nmt():
    path_to_zip = tf.keras.utils.get_file(
    'spa-eng.zip', origin='http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip',
    extract=True)

    path_to_file = os.path.dirname(path_to_zip)+"/spa-eng/spa.txt"
    return path_to_file


### Define a NMTDataset class with necessary functions to follow Step 1 to Step 4. 
The ```call()``` will return:
1. ```train_dataset```  and ```val_dataset``` : ```tf.data.Dataset``` objects
2. ```inp_lang_tokenizer``` and ```targ_lang_tokenizer``` : ```tf.keras.preprocessing.text.Tokenizer``` objects 

In [ ]:
class NMTDataset:
    def __init__(self, problem_type='en-spa'):
        self.problem_type = 'en-spa'
        self.inp_lang_tokenizer = None
        self.targ_lang_tokenizer = None
    

    def unicode_to_ascii(self, s):
        return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

    ## Step 1 and Step 2 
    def preprocess_sentence(self, w):
        w = self.unicode_to_ascii(w.lower().strip())

        # creating a space between a word and the punctuation following it
        # eg: "he is a boy." => "he is a boy ."
        # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
        w = re.sub(r"([?.!,¿])", r" \1 ", w)
        w = re.sub(r'[" "]+', " ", w)

        # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
        w = re.sub(r"[^ก-๛ะัาำิีึืฺุูเแโใไๅๆ็่้๊๋์a-zA-Z?.!,¿]+", " ", w)

        w = w.strip()

        # adding a start and an end token to the sentence
        # so that the model know when to start and stop predicting.
        w = '<start> ' + w + ' <end>'
        return w
    
    def create_dataset(self, path, num_examples):
        # path : path to spa-eng.txt file
        # num_examples : Limit the total number of training example for faster training (set num_examples = len(lines) to use full data)
        # lines = io.open(path, encoding='UTF-8').read().strip().split('\n')
        lines = seq_lst
        for i in range(1000, 1100):
          print(lines[i])
        word_pairs = [[self.preprocess_sentence(w) for w in l.split('\t')]  for l in lines[:num_examples]]

        return zip(*word_pairs)

    # Step 3 and Step 4
    def tokenize(self, lang):
        # lang = list of sentences in a language
        
        # print(len(lang), "example sentence: {}".format(lang[0]))
        lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='', oov_token='<OOV>')
        lang_tokenizer.fit_on_texts(lang)

        ## tf.keras.preprocessing.text.Tokenizer.texts_to_sequences converts string (w1, w2, w3, ......, wn) 
        ## to a list of correspoding integer ids of words (id_w1, id_w2, id_w3, ...., id_wn)
        tensor = lang_tokenizer.texts_to_sequences(lang) 

        ## tf.keras.preprocessing.sequence.pad_sequences takes argument a list of integer id sequences 
        ## and pads the sequences to match the longest sequences in the given input
        tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')

        return tensor, lang_tokenizer

    def load_dataset(self, path, num_examples=None):
        # creating cleaned input, output pairs
        targ_lang, inp_lang = self.create_dataset(path, num_examples)

        input_tensor, inp_lang_tokenizer = self.tokenize(inp_lang)
        target_tensor, targ_lang_tokenizer = self.tokenize(targ_lang)

        return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

    def call(self, num_examples, BUFFER_SIZE, BATCH_SIZE):
        file_path = download_nmt()
        input_tensor, target_tensor, self.inp_lang_tokenizer, self.targ_lang_tokenizer = self.load_dataset(file_path, num_examples)
        
        input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

        train_dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train))
        train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

        val_dataset = tf.data.Dataset.from_tensor_slices((input_tensor_val, target_tensor_val))
        val_dataset = val_dataset.batch(BATCH_SIZE, drop_remainder=True)

        return train_dataset, val_dataset, self.inp_lang_tokenizer, self.targ_lang_tokenizer

In [ ]:
BUFFER_SIZE = 32000
BATCH_SIZE = 64
# Let's limit the #training examples for faster training
num_examples = 42890

dataset_creator = NMTDataset('en-spa')
train_dataset, val_dataset, inp_lang, targ_lang = dataset_creator.call(num_examples, BUFFER_SIZE, BATCH_SIZE)

2654208/2638744 [==============================] - 0s 0us/step
30 / 06 / 2019 เรา มา เที่ยว ทะเล ด้วยกัน ครั้งแรก เค้า ถ่ายรูป เธอ ไว้ เยอะ มาก ๆ พอ มาถึง วัน แรก เธอ ก็ วิ่ง เข้าหา ทะเล เลย พร้อมทั้ง ถือ ถุง ขนม ไป ด้วย พอ ขนมเปียก เธอ ก็ เสียดาย น้อง ไม่ น่า ทํา น้อง เปียก เลย พอ วัน ที่สอง เรา ไปดู แมว น้ํา กัน เธอ ร้อง อุ๋งๆ ดัง กว่า แมว น้ํา ทั้ง ฝูง อีก รู้ตัว ม้า ย มัน น่ารัก นัก นะ ยัย คน นี้ ฮึ่ย !	30 / 06 / 2019 เรา มา เที่ยว ทะเล ด้วยกัน ครั้งแรก เขา ถ่ายรูป เธอ ไว้ เยอะ มาก ฟ พอ มาถึง วัน แรก เธอ ก็ วิ่ง เข้าหา ทะเล เลย พร้อมทั้ง ถือ ถุง ขนม ไป ด้วย พอ ขนมเปียก เธอ ก็ เสียดาย น้อง ไม่ น่า ทํา น้อง เปียก เลย พอ วัน ที่สอง เรา ไปดู แมว น้ํา กัน เธอ ร้อง อุ๋ง ๆ ดัง กว่า แมว น้ํา ทั้ง ฝูง อีก รู้ตัว ไหม มัน น่ารัก นัก นะ ยาย คน นี้ ฮึ่ย !
เน็ต บ้าน เน่า อีก ละ ยี่ห้อ ทอ ทหาร ใน ตํา นาน ทั้งๆ ที่ ก่อนหน้านี้ พัง ติดกัน 3 วัน จน ต้อง เปลี่ยน router เปลี่ยน ไม่ กี่ เดือน พัง อีกแล้ว จ้า ช่วง พัง คือ หัก เงิน ไป 3 วัน แต่ ก็ คือ ต้อง ใช้ เน็ต มือถือ ต่อ เปลือง ไป อีก ตะกี้ โทร ไป โ

In [ ]:
example_input_batch, example_target_batch = next(iter(train_dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([64, 215]), TensorShape([64, 216]))

### Some important parameters

In [ ]:
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1
max_length_input = example_input_batch.shape[1]
max_length_output = example_target_batch.shape[1]

embedding_dim = 256
units = 1024
steps_per_epoch = num_examples//BATCH_SIZE


In [ ]:
print("max_length_english, max_length_spanish, vocab_size_english, vocab_size_spanish")
max_length_input, max_length_output, vocab_inp_size, vocab_tar_size

max_length_english, max_length_spanish, vocab_size_english, vocab_size_spanish


(215, 216, 33827, 35757)

In [ ]:
##### 

class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)

    ##-------- LSTM layer in Encoder ------- ##
    self.lstm_layer = tf.keras.layers.LSTM(self.enc_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    


  def call(self, x, hidden):
    x = self.embedding(x)
    output, h, c = self.lstm_layer(x, initial_state = hidden)
    return output, h, c

  def initialize_hidden_state(self):
    return [tf.zeros((self.batch_sz, self.enc_units)), tf.zeros((self.batch_sz, self.enc_units))] 

In [ ]:
## Test Encoder Stack

encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)


# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_h, sample_c = encoder(example_input_batch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder h vecotr shape: (batch size, units) {}'.format(sample_h.shape))
print ('Encoder c vector shape: (batch size, units) {}'.format(sample_c.shape))

Encoder output shape: (batch size, sequence length, units) (64, 215, 1024)
Encoder h vecotr shape: (batch size, units) (64, 1024)
Encoder c vector shape: (batch size, units) (64, 1024)


In [ ]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz, attention_type='luong'):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.attention_type = attention_type
    
    # Embedding Layer
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    
    #Final Dense layer on which softmax will be applied
    self.fc = tf.keras.layers.Dense(vocab_size)

    # Define the fundamental cell for decoder recurrent structure
    self.decoder_rnn_cell = tf.keras.layers.LSTMCell(self.dec_units)
   


    # Sampler
    self.sampler = tfa.seq2seq.sampler.TrainingSampler()

    # Create attention mechanism with memory = None
    self.attention_mechanism = self.build_attention_mechanism(self.dec_units, 
                                                              None, self.batch_sz*[max_length_input], self.attention_type)

    # Wrap attention mechanism with the fundamental rnn cell of decoder
    self.rnn_cell = self.build_rnn_cell(batch_sz)

    # Define the decoder with respect to fundamental rnn cell
    self.decoder = tfa.seq2seq.BasicDecoder(self.rnn_cell, sampler=self.sampler, output_layer=self.fc)

    
  def build_rnn_cell(self, batch_sz):
    rnn_cell = tfa.seq2seq.AttentionWrapper(self.decoder_rnn_cell, 
                                  self.attention_mechanism, attention_layer_size=self.dec_units)
    return rnn_cell

  def build_attention_mechanism(self, dec_units, memory, memory_sequence_length, attention_type='luong'):
    # ------------- #
    # typ: Which sort of attention (Bahdanau, Luong)
    # dec_units: final dimension of attention outputs 
    # memory: encoder hidden states of shape (batch_size, max_length_input, enc_units)
    # memory_sequence_length: 1d array of shape (batch_size) with every element set to max_length_input (for masking purpose)

    if(attention_type=='bahdanau'):
      return tfa.seq2seq.BahdanauAttention(units=dec_units, memory=memory, memory_sequence_length=memory_sequence_length)
    else:
      return tfa.seq2seq.LuongAttention(units=dec_units, memory=memory, memory_sequence_length=memory_sequence_length)

  def build_initial_state(self, batch_sz, encoder_state, Dtype):
    decoder_initial_state = self.rnn_cell.get_initial_state(batch_size=batch_sz, dtype=Dtype)
    decoder_initial_state = decoder_initial_state.clone(cell_state=encoder_state)
    return decoder_initial_state


  def call(self, inputs, initial_state):
    x = self.embedding(inputs)
    outputs, _, _ = self.decoder(x, initial_state=initial_state, sequence_length=self.batch_sz*[max_length_output-1])
    return outputs


In [ ]:
# Test decoder stack

decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE, 'luong')
sample_x = tf.random.uniform((BATCH_SIZE, max_length_output))
decoder.attention_mechanism.setup_memory(sample_output)
initial_state = decoder.build_initial_state(BATCH_SIZE, [sample_h, sample_c], tf.float32)


sample_decoder_outputs = decoder(sample_x, initial_state)

print("Decoder Outputs Shape: ", sample_decoder_outputs.rnn_output.shape)


Decoder Outputs Shape:  (64, 215, 35757)


## Define the optimizer and the loss function

In [ ]:
optimizer = tf.keras.optimizers.Adam()


def loss_function(real, pred):
  # real shape = (BATCH_SIZE, max_length_output)
  # pred shape = (BATCH_SIZE, max_length_output, tar_vocab_size )
  cross_entropy = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')
  loss = cross_entropy(y_true=real, y_pred=pred)
  mask = tf.logical_not(tf.math.equal(real,0))   #output 0 for y=0 else output 1
  mask = tf.cast(mask, dtype=loss.dtype)  
  loss = mask* loss
  loss = tf.reduce_mean(loss)
  return loss  

## Checkpoints (Object-based saving)

In [ ]:
checkpoint_dir = './drive/MyDrive/AIBuilders/seq_checkpoint'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

## One train_step operations

In [ ]:
@tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_output, enc_h, enc_c = encoder(inp, enc_hidden)


    dec_input = targ[ : , :-1 ] # Ignore <end> token
    real = targ[ : , 1: ]         # ignore <start> token

    # Set the AttentionMechanism object with encoder_outputs
    decoder.attention_mechanism.setup_memory(enc_output)

    # Create AttentionWrapperState as initial_state for decoder
    decoder_initial_state = decoder.build_initial_state(BATCH_SIZE, [enc_h, enc_c], tf.float32)
    pred = decoder(dec_input, decoder_initial_state)
    logits = pred.rnn_output
    loss = loss_function(real, logits)

  variables = encoder.trainable_variables + decoder.trainable_variables
  gradients = tape.gradient(loss, variables)
  optimizer.apply_gradients(zip(gradients, variables))

  return loss

## Train the model

In [ ]:
EPOCHS = 40

for epoch in range(EPOCHS):
  start = time.time()

  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0
  # print(enc_hidden[0].shape, enc_hidden[1].shape)

  for (batch, (inp, targ)) in enumerate(tqdm(train_dataset.take(steps_per_epoch))):
    batch_loss = train_step(inp, targ, enc_hidden)
    total_loss += batch_loss

    if batch % 100 == 0:
      print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                   batch,
                                                   batch_loss.numpy()))
  # saving (checkpoint) the model every 2 epochs
  if (epoch + 1) % 5 == 0:
    checkpoint.save(file_prefix = checkpoint_prefix)

  print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
  print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

  0%|          | 1/375 [00:12<1:16:09, 12.22s/it]

Epoch 1 Batch 0 Loss 2.2727


 27%|██▋       | 101/375 [05:09<13:46,  3.02s/it]

Epoch 1 Batch 100 Loss 2.2532


 54%|█████▎    | 201/375 [10:06<08:47,  3.03s/it]

...

Epoch 40 Batch 200 Loss nan


 80%|████████  | 301/375 [14:52<03:42,  3.01s/it]

Epoch 40 Batch 300 Loss nan


100%|██████████| 375/375 [18:32<00:00,  2.97s/it]


Epoch 40 Loss nan
Time taken for 1 epoch 1116.2200396060944 sec



## Use tf-addons BasicDecoder for decoding


In [ ]:
def evaluate_sentence(sentence):
  sentence = dataset_creator.preprocess_sentence(sentence)
  inputs = []
  for i in sentence.split(' '):
    try:
      a = inp_lang.word_index[i]
    except:
      inputs.append(1)
    else:
      inputs.append(a)
  # inputs = [inp_lang.word_index[i] for i in sentence.split(' ')]
  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                          maxlen=max_length_input,
                                                          padding='post')
  inputs = tf.convert_to_tensor(inputs)
  inference_batch_size = inputs.shape[0]
  result = ''

  enc_start_state = [tf.zeros((inference_batch_size, units)), tf.zeros((inference_batch_size,units))]
  enc_out, enc_h, enc_c = encoder(inputs, enc_start_state)

  dec_h = enc_h
  dec_c = enc_c

  start_tokens = tf.fill([inference_batch_size], targ_lang.word_index['<start>'])
  end_token = targ_lang.word_index['<end>']

  greedy_sampler = tfa.seq2seq.GreedyEmbeddingSampler()

  # Instantiate BasicDecoder object
  decoder_instance = tfa.seq2seq.BasicDecoder(cell=decoder.rnn_cell, sampler=greedy_sampler, output_layer=decoder.fc)
  # Setup Memory in decoder stack
  decoder.attention_mechanism.setup_memory(enc_out)

  # set decoder_initial_state
  decoder_initial_state = decoder.build_initial_state(inference_batch_size, [enc_h, enc_c], tf.float32)


  ### Since the BasicDecoder wraps around Decoder's rnn cell only, you have to ensure that the inputs to BasicDecoder 
  ### decoding step is output of embedding layer. tfa.seq2seq.GreedyEmbeddingSampler() takes care of this. 
  ### You only need to get the weights of embedding layer, which can be done by decoder.embedding.variables[0] and pass this callabble to BasicDecoder's call() function

  decoder_embedding_matrix = decoder.embedding.variables[0]
  
  outputs, _, _ = decoder_instance(decoder_embedding_matrix, start_tokens = start_tokens, end_token= end_token, initial_state=decoder_initial_state)
  return outputs.sample_id.numpy()

def translate(sentence):
  result = evaluate_sentence(sentence)
  # print(result)
  result = targ_lang.sequences_to_texts(result)
  return result
  # print('Input: %s' % (sentence))
  # print('Predicted translation: {}'.format(result))

## Restore the latest checkpoint and test

In [ ]:
# restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [ ]:
translate(u'เรา มา เที่ยว ทะเล ด้วยกัน ครั้งแรก เค้า ถ่ายรูป เธอ ไว้ เยอะ มาก ๆ พอ มาถึง วัน แรก เธอ ก็ วิ่ง เข้าหา ทะเล เลย พร้อมทั้ง ถือ ถุง ขนม ไป ด้วย พอ ขนมเปียก เธอ ก็ เสียดาย น้อง ไม่ น่า ทํา น้อง เปียก')

['ไม ไม ท ท ท ท ท ท ไม ไม ไม ไม ไม ไม ไม ไม ไม ไม ก ไม ก ไม ก ไม ก ไม ก ก ก ก ก ก ก ก ก ก ก ก ก ก ก ก ก ก ก ก ก ก ก ก ก ก ก ก ก ก ก ก ก ก ก ก ก ก ก ก ก <end>']

In [ ]:
seq_lst

['happy birthday พี่ เมี๊ยว ผู้ อํา นวย การสมาคม ฟ้า สี รุ้ง แห่ง ประเทศ ไทย .. ตั้งแต่ เรา รู้จัก กัน ใน วงการ hiv / aids นั้น พีท ได้รับ ความรัก ความเมตตา จาก พี่ เมี๊ยว และ พี่ ๆ ใน สมาคม ฟ้า สี รุ้ง แห่ง ประเทศ ไทย เสมอมา ... ขอบคุณ สํา หรับ การผลักดัน ให้ มีระบบ บริการ สุขภาพ ดี ดี ให้ เกิดขึ้น ในประเทศ ไทย at สมาคม ฟ้า สี รุ้ง แห่ง ประเทศ ไทย สํา นัก งานใหญ่ ราม คํา แหง rainbow sky association of thailand head office\thappy birthday พี่ เมี๊ยว ผู้ อํา นวย การสมาคม ฟ้า สี รุ้ง แห่ง ประเทศ ไทย .. ตั้งแต่ เรา รู้จัก กัน ใน วงการ hiv / aids นั้น พีท ได้รับ ความรัก ความเมตตา จาก พี่ เมี๊ยว และ พี่ ๆ ใน สมาคม ฟ้า สี รุ้ง แห่ง ประเทศ ไทย เสมอมา ... ขอบคุณ สํา หรับ การผลักดัน ให้ มีระบบ บริการ สุขภาพ ดี ๆ ให้ เกิดขึ้น ในประเทศ ไทย at สมาคม ฟ้า สี รุ้ง แห่ง ประเทศ ไทย สํา นัก งานใหญ่ ราม คํา แหง rainbow sky association of thailand head office',
 'รีวิว # thequake วินาศภัย แห่ง แผ่นดินไหว ใน นอร์เวย์ หนัง สํา หรับ คน ชอบ สาย ภัยพิบัติ ให้ 6 / 10 ที่มา ของ พล็อต เรื่อง เยี่ยม ต้นเรื่อง แอบ 

In [ ]:
NUM_SAMPLE = 1000
cer_higher = 0
cer_lower = 0
cer_equal = 0
total_cer_original = 0
total_cer_predicted = 0

for i in tqdm(range(NUM_SAMPLE)):
  input_txt = seq_lst[i].split("\t")
  original_txt = input_txt[0]
  label_txt = input_txt[1]
  cer_original = cer(original_txt, label_txt)
  total_cer_original += cer_original

  predicted_txt = translate(original_txt)[0]
  cer_predicted = cer(predicted_txt, label_txt)
  total_cer_predicted += cer_predicted
  # print(original_txt)
  # print(predicted_txt)
  # print(f"CER: {cer_original}, {cer_predicted}")

  if cer_predicted > cer_original:
    cer_higher += 1
  elif cer_predicted < cer_original:
    cer_lower += 1
  elif cer_predicted == cer_original:
    cer_equal += 1
print(f"AVG CER ORIGINAL: {float(total_cer_original/NUM_SAMPLE)}")
print(f"AVG CER PREDICTED: {float(total_cer_predicted/NUM_SAMPLE)}")
print(f"# HIGHER CER PREDICTION: {cer_higher}")
print(f"# LOWER CER PREDICTION: {cer_lower}")
print(f"# EQUAL CER PREDICTION: {cer_equal}")

100%|██████████| 1000/1000 [08:17<00:00,  2.01it/s]

AVG CER ORIGINAL: 0.02363859544814001
AVG CER PREDICTED: 1.864385943346359
# HIGHER CER PREDICTION: 1000
# LOWER CER PREDICTION: 0
# EQUAL CER PREDICTION: 0


## Use tf-addons BeamSearchDecoder 


In [ ]:
def beam_evaluate_sentence(sentence, beam_width=3):
  sentence = dataset_creator.preprocess_sentence(sentence)
  inputs = []
  for i in sentence.split(' '):
    try:
      a = inp_lang.word_index[i]
    except:
      inputs.append(1)
    else:
      inputs.append(a)
  # inputs = [inp_lang.word_index[i] for i in sentence.split(' ')]
  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                          maxlen=max_length_input,
                                                          padding='post')
  inputs = tf.convert_to_tensor(inputs)
  inference_batch_size = inputs.shape[0]
  result = ''

  enc_start_state = [tf.zeros((inference_batch_size, units)), tf.zeros((inference_batch_size,units))]
  enc_out, enc_h, enc_c = encoder(inputs, enc_start_state)

  dec_h = enc_h
  dec_c = enc_c

  start_tokens = tf.fill([inference_batch_size], targ_lang.word_index['<start>'])
  end_token = targ_lang.word_index['<end>']

  # From official documentation
  # NOTE If you are using the BeamSearchDecoder with a cell wrapped in AttentionWrapper, then you must ensure that:
  # The encoder output has been tiled to beam_width via tfa.seq2seq.tile_batch (NOT tf.tile).
  # The batch_size argument passed to the get_initial_state method of this wrapper is equal to true_batch_size * beam_width.
  # The initial state created with get_initial_state above contains a cell_state value containing properly tiled final state from the encoder.

  enc_out = tfa.seq2seq.tile_batch(enc_out, multiplier=beam_width)
  decoder.attention_mechanism.setup_memory(enc_out)
  print("beam_with * [batch_size, max_length_input, rnn_units] :  3 * [1, 16, 1024]] :", enc_out.shape)

  # set decoder_inital_state which is an AttentionWrapperState considering beam_width
  hidden_state = tfa.seq2seq.tile_batch([enc_h, enc_c], multiplier=beam_width)
  decoder_initial_state = decoder.rnn_cell.get_initial_state(batch_size=beam_width*inference_batch_size, dtype=tf.float32)
  decoder_initial_state = decoder_initial_state.clone(cell_state=hidden_state)

  # Instantiate BeamSearchDecoder
  decoder_instance = tfa.seq2seq.BeamSearchDecoder(decoder.rnn_cell,beam_width=beam_width, output_layer=decoder.fc)
  decoder_embedding_matrix = decoder.embedding.variables[0]

  # The BeamSearchDecoder object's call() function takes care of everything.
  outputs, final_state, sequence_lengths = decoder_instance(decoder_embedding_matrix, start_tokens=start_tokens, end_token=end_token, initial_state=decoder_initial_state)
  # outputs is tfa.seq2seq.FinalBeamSearchDecoderOutput object. 
  # The final beam predictions are stored in outputs.predicted_id
  # outputs.beam_search_decoder_output is a tfa.seq2seq.BeamSearchDecoderOutput object which keep tracks of beam_scores and parent_ids while performing a beam decoding step
  # final_state = tfa.seq2seq.BeamSearchDecoderState object.
  # Sequence Length = [inference_batch_size, beam_width] details the maximum length of the beams that are generated

  
  # outputs.predicted_id.shape = (inference_batch_size, time_step_outputs, beam_width)
  # outputs.beam_search_decoder_output.scores.shape = (inference_batch_size, time_step_outputs, beam_width)
  # Convert the shape of outputs and beam_scores to (inference_batch_size, beam_width, time_step_outputs)
  final_outputs = tf.transpose(outputs.predicted_ids, perm=(0,2,1))
  beam_scores = tf.transpose(outputs.beam_search_decoder_output.scores, perm=(0,2,1))
  
  return final_outputs.numpy(), beam_scores.numpy()

In [ ]:
def beam_translate(sentence):
  result, beam_scores = beam_evaluate_sentence(sentence)
  print(result.shape, beam_scores.shape)
  for beam, score in zip(result, beam_scores):
    print(beam.shape, score.shape)
    output = targ_lang.sequences_to_texts(beam)
    output = [a[:a.index('<end>')] for a in output]
    beam_score = [a.sum() for a in score]
    print('Input: %s' % (sentence))
    for i in range(len(output)):
      print('{} Predicted translation: {}  {}'.format(i+1, output[i], beam_score[i]))


In [ ]:
beam_translate(u'เรา มา เที่ยว ทะเล ด้วยกัน ครั้งแรก เค้า ถ่ายรูป เธอ ไว้ เยอะ มาก ๆ พอ มาถึง วัน แรก เธอ ก็ วิ่ง เข้าหา ทะเล เลย พร้อมทั้ง ถือ ถุง ขนม ไป ด้วย พอ ขนมเปียก เธอ ก็ เสียดาย น้อง ไม่ น่า ทํา น้อง เปียก')

beam_with * [batch_size, max_length_input, rnn_units] :  3 * [1, 16, 1024]] : (3, 215, 1024)


/usr/local/lib/python3.7/dist-packages/tensorflow_addons/utils/resource_loader.py:103: UserWarning: You are currently using TensorFlow 2.8.2 and trying to load a custom op (custom_ops/seq2seq/_beam_search_ops.so).
TensorFlow Addons has compiled its custom ops against TensorFlow 2.2.0, and there are no compatibility guarantees between the two versions. 
This means that you might get segfaults when loading the custom op, or other kind of low-level errors.
 If you do, do not file an issue on Github. This is a known limitation.

It might help you to fallback to pure Python ops with TF_ADDONS_PY_OPS . To do that, see https://github.com/tensorflow/addons#gpucpu-custom-ops 

You can also change the TensorFlow version installed on your system. You would need a TensorFlow version equal to or above 2.2.0 and strictly below 2.3.0.
 Note that nightly versions of TensorFlow, as well as non-pip TensorFlow like `conda install tensorflow` or compiled from source are not supported.

The last solution i

(1, 3, 57) (1, 3, 57)
(3, 57) (3, 57)
Input: เรา มา เที่ยว ทะเล ด้วยกัน ครั้งแรก เค้า ถ่ายรูป เธอ ไว้ เยอะ มาก ๆ พอ มาถึง วัน แรก เธอ ก็ วิ่ง เข้าหา ทะเล เลย พร้อมทั้ง ถือ ถุง ขนม ไป ด้วย พอ ขนมเปียก เธอ ก็ เสียดาย น้อง ไม่ น่า ทํา น้อง เปียก
1 Predicted translation: รวว นายชาง น ท ท ท ท ท ท ท ท ท ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม   -6177.689453125
2 Predicted translation: รวว นายชาง น ท ท ท ท ท ท ท ท ท ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ก ม ม ม   -6183.77880859375
3 Predicted translation: รวว นายชาง น ท ท ท ท ท ท ท ท ท ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ม ก ม ม   -6186.3642578125
